In [40]:
import pandas as pd
import numpy as np

from pymongo import MongoClient

import re

#fixed issue with ascii encoding errors
import sys

reload(sys)
sys.setdefaultencoding("utf-8")
#TODO: countvectorizer (ignores punctuation), nltk tokenizer

from nltk.tokenize import *

In [41]:
client = MongoClient()
client = MongoClient('localhost', 27017)
db = client.POETRY
poems = db.POEM
rawpoem = pd.DataFrame(list(poems.find()))

some poem videos made it in the data so we need to remove them

In [42]:
coltouse = ['poem_link', 'poet_poems_url','poem_yrpub','poem_title','poem_text']
# poemdf = rawpoem.loc[rawpoem['poem_title'] ][coltouse]

poemdf = rawpoem.loc[rawpoem['poem_link'].str.contains('video') == False][coltouse]

In [105]:
text = poemdf.poem_text[1]

In [100]:
expression = '.+\n'

len(re.findall(expression, text))

11

In [99]:
text = u''.join(text)

In [106]:
# poemdf['poem_text'].map(lambda x: len(re.findall(expression,u''.join(x))))
len(text)

2

In [153]:
def countlines(text):
    expression ='.+\n'
    strtext = u''.join(text)
    if len(re.findall(expression,strtext)) == 0:
        numnewl = len(text)
    else:
        numnewl = len(re.findall(expression, strtext))
    return float(numnewl)

def countstanza(text):
    excexpression = '.+\n'
    expression = '.+\n\n'
    strtext = u''.join(text)
    numstanzas = 0
    if len(re.findall(excexpression,strtext)) == 0:
        for i in text:
            if len(str(i)) ==0:
                numstanzas += 1
    else:
        numstanzas = len(re.findall(expression, strtext))
    return float(numstanzas+1)

In [124]:
poemdf['numlines'] = poemdf['poem_text'].apply(lambda x: countlines(x))
poemdf['numstanzas'] = poemdf['poem_text'].apply(lambda x: countstanza(x))

In [143]:
poemdf.numstanzas.mean()
poemdf.groupby(['poet_poems_url'])['numstanzas'].mean()

poet_poems_url
http://www.poets.org/poetsorg/poems/120970     1.000000
http://www.poets.org/poetsorg/poems/120979     1.000000
http://www.poets.org/poetsorg/poems/120986     1.000000
http://www.poets.org/poetsorg/poems/121506     1.000000
http://www.poets.org/poetsorg/poems/123231     1.000000
http://www.poets.org/poetsorg/poems/125236     1.000000
http://www.poets.org/poetsorg/poems/129086     2.000000
http://www.poets.org/poetsorg/poems/339926     1.000000
http://www.poets.org/poetsorg/poems/404586     1.000000
http://www.poets.org/poetsorg/poems/405236     1.000000
http://www.poets.org/poetsorg/poems/406436     1.000000
http://www.poets.org/poetsorg/poems/417266     1.000000
http://www.poets.org/poetsorg/poems/44303      3.842105
http://www.poets.org/poetsorg/poems/44311      3.000000
http://www.poets.org/poetsorg/poems/44328      1.000000
http://www.poets.org/poetsorg/poems/44330     10.500000
http://www.poets.org/poetsorg/poems/44348     11.666667
http://www.poets.org/poetsorg/poe

In [208]:
text = poemdf.poem_text[8]
text = u''.join(text)

In [163]:
def sent_avglines(text):
    nlines = []
    t_sent = sent_tokenize(text)
    for i in t_sent:
        nlines.append(countlines(i))
    return np.mean(nlines)
# sent_tokenize(text)

In [170]:
poemdf['avgline_stanza'] = poemdf['numlines']/poemdf['numstanzas']

In [188]:
def count_words(text, distinct = False):    
    tokenizer = RegexpTokenizer(r'\w+')
    tokenized = tokenizer.tokenize(u''.join(text))
    numwords = len(tokenized)
    if distinct:
        numwords = len(list(set(tokenized)))
    return numwords            

In [213]:
poemdf['wordcount'] = poemdf['poem_text'].apply(lambda x: count_words(x))
poemdf['wordcount_d'] = poemdf['poem_text'].apply(lambda x: count_words(x,True))
poemdf['ttr'] = poemdf['wordcount_d']/poemdf['wordcount']

In [276]:
poemdf.head()
poemdf.groupby(['poem_yrpub'])['ttr'].mean()
# poemdf.dtypes

poem_yrpub
\n                0.845390
1556              0.774497
1710              0.796512
1717              0.389560
1733              0.492512
1734              0.836364
1735              0.592233
1772              0.764577
1773              0.705882
1775              0.711246
1909              0.776128
1910              0.584582
1912              0.696538
1913              0.713280
1914              0.559258
1915              0.675723
1916              0.684400
1917              0.735873
1918              0.627521
1919              0.757061
1920              0.722060
1921              0.755508
1922              0.698945
1923              0.746556
1924              0.857143
1926              1.000000
1927              0.503872
1928              0.700000
1931              0.734234
1933              0.780089
                    ...   
1986              0.655750
1987              0.654762
1988              0.631535
1989              0.655286
1990              0.659660
1991             

In [264]:
# poem.poem_yrpub
poemdf['poem_yrpub'].value_counts()[50:80]
# poemdf['yrpub'] = poemdf['poem_yrpub'].map(lambda x: re.findall(r'[\d]+', x))
# poemdf['yrpub'] = poemdf['yrpub'].astype(int)
yrpub = poemdf['poem_yrpub'].apply(pd.Series)

In [267]:
yrpub.columns = ['yrpublished']

In [268]:
poemdf = pd.concat([poemdf, yrpub], axis = 1)

In [201]:
from nltk.corpus import stopwords
from nltk import *

In [211]:
text1 = word_tokenize(text)
pos_tag(text1)

[(u'I', 'PRP'),
 (u"'m", 'VBP'),
 (u'calling', 'VBG'),
 (u'out', 'RP'),
 (u'from', 'IN'),
 (u'pictures', 'NNS'),
 (u'to', 'TO'),
 (u'your', 'PRP$'),
 (u'vision', 'NN'),
 (u'creating', 'VBG'),
 (u'itturn', 'NN'),
 (u'right', 'RB'),
 (u',', ','),
 (u'that', 'IN'),
 (u'dream', 'NN'),
 (u'building', 'NN'),
 (u'cutglass', 'NN'),
 (u'window', 'NN'),
 (u'in', 'IN'),
 (u'door.Automatically', 'JJ'),
 (u'inside', 'NN'),
 (u'their', 'PRP$'),
 (u'apartment', 'NN'),
 (u',', ','),
 (u'you', 'PRP'),
 (u'do', 'VBP'),
 (u"n't", 'RB'),
 (u'haveto', 'VB'),
 (u'get', 'NN'),
 (u'there', 'RB'),
 (u'.', '.'),
 (u'This', 'DT'),
 (u'is', 'VBZ'),
 (u'before', 'IN'),
 (u'the', 'DT'),
 (u'lost', 'NN'),
 (u'sacred', 'VBD'),
 (u'corpus', 'NN'),
 (u'vision', 'NN'),
 (u',', ','),
 (u'someone', 'NN'),
 (u'says', 'VBZ'),
 (u'Look', 'NNP'),
 (u'at', 'IN'),
 (u'my', 'PRP$'),
 (u'author', 'NN'),
 (u'photo', 'NN'),
 (u'.', '.'),
 (u'Ido', 'NNP'),
 (u"n't", 'RB'),
 (u'really', 'RB'),
 (u'want', 'VB'),
 (u'to', 'TO'),
 (u'I'